https://colab.research.google.com/drive/1P9iQHtUH5KUbTVtp8B4-AopZzEEPE0lw?usp=sharing#scrollTo=4YS7XI2bZTyz

https://github.com/aub-mind/arabert

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install optuna==2.3.0
!pip -q install transformers==4.2.1
!pip -q install farasapy
!pip -q install pyarabic
# !git clone https://huggingface.co/qarib/bert-base-qarib

     |████████████████████████████████| 258 kB 8.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 80 kB 10.7 MB/s 
     |████████████████████████████████| 164 kB 76.0 MB/s 
     |████████████████████████████████| 75 kB 5.4 MB/s 
     |████████████████████████████████| 111 kB 68.6 MB/s 
     |████████████████████████████████| 141 kB 58.5 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
     |████████████████████████████████| 1.8 MB 7.1 MB/s 
     |████████████████████████████████| 2.9 MB 55.6 MB/s 
     |████████████████████████████████| 895 kB 62.3 MB/s 
     |████████████████████████████████| 126 kB 8.3 MB/s 


In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [4]:
hard_map = { 0 : 'fake', 1 : 'real', }

df_train = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2train_news.csv", header=0,encoding='utf-8')
df_train.label= df_train.label.apply(lambda x: hard_map[x])

df_val = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2val_news.csv", header=0,encoding='utf-8')
df_val.label= df_val.label.apply(lambda x: hard_map[x])

label_list_HARD = ['fake', 'real']




# Trainer

In [5]:
# from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [6]:
dataset_name = 'NEWS'
model_name = 'UBC-NLP/ARBERT'
task_name = 'classification'
max_len = 512

In [7]:
# arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

# df_train.text = df_train.text.apply(lambda x:   arabert_prep.preprocess(x))
# df_val.text = df_val.text.apply(lambda x:   arabert_prep.preprocess(x))  

In [8]:
class BERTDataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

# 

In [9]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(df_train.text.to_list(),df_train.label.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(df_val.text.to_list()  ,df_val.label.to_list()  ,model_name,  max_len, label_map)


{'fake': 0, 'real': 1}


In [10]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [11]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# Regular Training

In [12]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8

steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

160
1280


In [13]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

Some weights of the model checkpoint at UBC-NLP/ARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UB

In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.052746,0.980418,0.984962,0.976062,0.985959,19.781800,64.807000
1,No log,0.046632,0.985010,0.981813,0.988309,0.989080,19.960900,64.226000
2,No log,0.080411,0.986096,0.982378,0.989953,0.989860,19.899000,64.425000
3,0.051100,0.064487,0.987093,0.986001,0.988198,0.990640,19.866200,64.532000
4,0.051100,0.049478,0.990309,0.989756,0.990865,0.992980,19.936300,64.305000
5,0.051100,0.050068,0.990309,0.989756,0.990865,0.992980,19.865900,64.533000
6,0.000600,0.050315,0.990309,0.989756,0.990865,0.992980,19.995400,64.115000
7,0.000600,0.050360,0.990309,0.989756,0.990865,0.992980,20.002100,64.093000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:911: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,


              precision    recall  f1-score   support

           0       0.98      0.96      0.97       304
           1       0.99      0.99      0.99       978

    accuracy                           0.99      1282
   macro avg       0.98      0.98      0.98      1282
weighted avg       0.99      0.99      0.99      1282

[[291  13]
 [  5 973]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       304
           1       1.00      0.99      0.99       978

    accuracy                           0.99      1282
   macro avg       0.98      0.99      0.99      1282
weighted avg       0.99      0.99      0.99      1282

[[300   4]
 [ 10 968]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       304
           1       1.00      0.99      0.99       978

    accuracy                           0.99      1282
   macro avg       0.98      0.99      0.99      1282
weighted avg       0.99     

TrainOutput(global_step=1280, training_loss=0.02017243919253815, metrics={'train_runtime': 1913.9303, 'train_samples_per_second': 0.669, 'total_flos': 20518390727000064, 'epoch': 8.0})

In [16]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       304
           1       1.00      0.99      1.00       978

    accuracy                           0.99      1282
   macro avg       0.99      0.99      0.99      1282
weighted avg       0.99      0.99      0.99      1282

[[300   4]
 [  5 973]]


{'epoch': 8.0,
 'eval_accuracy': 0.9929797191887676,
 'eval_loss': 0.05035994201898575,
 'eval_macro_f1': 0.9903090471570937,
 'eval_macro_precision': 0.9897561957816668,
 'eval_macro_recall': 0.990864815412765,
 'eval_runtime': 20.0652,
 'eval_samples_per_second': 63.892}

# Test

In [17]:
df_test = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2test_news.csv")
print(df_test.label.value_counts())


1    1198
0     405
Name: label, dtype: int64


In [18]:
df_test.label= df_test.label.apply(lambda x: hard_map[x])

In [19]:
test_dataset = BERTDataset(df_test.text.to_list(),df_test.label.to_list(),model_name, max_len,label_map)

In [20]:
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       405
           1       0.99      1.00      1.00      1198

    accuracy                           0.99      1603
   macro avg       0.99      0.99      0.99      1603
weighted avg       0.99      0.99      0.99      1603

[[ 398    7]
 [   2 1196]]


{'epoch': 8.0,
 'eval_accuracy': 0.9943855271366189,
 'eval_loss': 0.03849562257528305,
 'eval_macro_f1': 0.9925357188128134,
 'eval_macro_precision': 0.9945906068162926,
 'eval_macro_recall': 0.9905233001504565,
 'eval_runtime': 25.0389,
 'eval_samples_per_second': 64.02}

# Predict

In [21]:
class Dataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [22]:

title_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/title_example.csv")
title_example.shape

count=0
for title in title_example.text:
   test_dataset = Dataset([title],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(test_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1

    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 0


In [23]:
text_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/text_example.csv")
text_example.shape
count=0

for text in text_example.text:
   text_dataset = Dataset([text],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(text_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1
    
print("Real : "+str(count))

[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[1]
[1]
Real : 6


In [24]:
# # Tokenize test data
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512) 
# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized) 
# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2) 
# # Define test trainer
# test_trainer = Trainer(model) 
# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset) 
# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

In [25]:
# print("===> Predicting...")
# trainer1 = Trainer(
#     model= model_init(),
#     args=training_args,
#     eval_dataset=test_dataset
# )
        
# results = {}
# result = trainer1.predict(test_dataset)

In [26]:
#  ----- 3. Predict -----#
# # Load test data
# test_data = pd.("drive/My Drive/Arabic_FakeNews/Model/Data/test_news.csv", header=0,encoding='utf-8')
# X_test = list(test_data["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)